In [ ]:
# Programmer: Joshua McAllister
# Date: 4-24-20
# Comments and additional functionality will come with time

from random import randint
from random import seed

def generate_member_list(category,
                         member_dict):
    member_list = []
    for member, member_tuple in member_dict.items():
        curr_member_data = [member, 0, 0]
        assigned_problem_dict = member_tuple[0]
        if (assigned_problem_dict):
            for assigned_category, assigned_category_dict in assigned_problem_dict.items():
                if category == assigned_category:
                    curr_member_data[1] = 1
                for assigned_difficulty, assigned_problem_set in assigned_category_dict.items():
                    curr_member_data[2] += (len(assigned_problem_set)*assigned_difficulty)
        member_list.append(curr_member_data)
    # Remove members which exceed the minimum weight
    member_list.sort(key=lambda x:x[2])
    min_member_weight = member_list[0][2]
    i = 1
    while (i < len(member_list)):
        if (member_list[i][2] > min_member_weight):
            member_list.pop(i)
        else:
            i += 1
    # If members have received problems from this category while others have not, remove those members
    parsed_member_list = []
    for member in member_list:
        if (member[1] == 0):
            parsed_member_list.append(member)
    if parsed_member_list:
        member_list = parsed_member_list
    return member_list

def distribute_problems(member_dict,
                        problem_dict):
    # Create a list of problems sorted by difficulty (descending)
    problem_list = []
    for category, problem_set_dict in problem_dict.items():
        category_list = []
        for difficulty, problem_set in problem_set_dict.items():
            category_list.append([category, difficulty, problem_set])
        problem_list.extend(category_list)
    problem_list.sort(key=lambda x:x[1], reverse=True)

    for batch in problem_list:
        category = batch[0]
        difficulty = batch[1]
        problem_set = batch[2]
        for problem in problem_set:
            # Generate current list of members with minimum weight & available problem distribution
            member_list = generate_member_list(category, member_dict)
            member_index = randint(0, len(member_list)-1)
            chosen_member = member_list[member_index][0]
            assigned_problem_dict = member_dict[chosen_member][0]
            if category not in assigned_problem_dict:
                assigned_problem_dict[category] = {}
            if difficulty not in assigned_problem_dict[category]:
                assigned_problem_dict[category][difficulty] = set()
            assigned_problem_dict[category][difficulty].add(problem)

def print_data(member_dict):
    for member, member_tuple in member_dict.items():
        print("---------------")
        print(member)
        assigned_problem_dict = member_tuple[0]
        for category, assigned_problem_dict in assigned_problem_dict.items():
            print_string = f"{category}: "
            problem_list = []
            for problem_set in assigned_problem_dict.values():
                for problem in problem_set:
                    problem_list.extend(problem.strip().split(','))
            problem_list.sort(key=int)
            print_string += str([int(problem) for problem in problem_list])
            print(print_string)
            
def parse_data(filename):
    member_dict = {}
    problem_dict = {}
    with open(filename) as file:
        for line in file:
            data = line.strip().split(',')
            # print(f"Current data: {data}")
            if (data[0] == "member"):
                member = data[1].strip("\"")
                if member not in member_dict:
                    member_dict[member] = ({}, 1)  # Handicap not currently implemented
                else:
                    print(f"ERROR: Received duplicate member information for {member}")
                    return
            elif (data[0] == "problem"):
                category = data[1].strip("\"")
                dict_info = data[2:]
                if category not in problem_dict:
                    problem_dict[category] = {}
                category_problem_dict = problem_dict[category]
                for info in dict_info:
                    info_list = info.strip().split(":")
                    difficulty = int(info_list[0])
                    problem_list_temp = info_list[1].strip("\"").replace(" ", ", ").split(".")
                    problem_list = []
                    # Attempt to find a range in the problem_set
                    range_list = problem_list_temp[0].strip().split("-")
                    if (len(range_list) == 2):
                        #print(f"Range list: {range_list}")
                        problem_list.extend([str(number) for number in range(int(range_list[0]), int(range_list[1])+1)])
                    else:
                        problem_list.extend(problem_list_temp)
                    category_problem_dict[difficulty] = set(problem_list)
            else:
                print(f"ERROR! data[0] = {data[0]}")
                return
    # print(member_dict)
    # print(problem_dict)
    distribute_problems(member_dict, problem_dict)
    print_data(member_dict)

In [ ]:
seed(a=0)   # None for randomness based on current system time, an int for fixed seed
filename = "input.csv"   # Relative to operating folder
parse_data(filename)